In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import json,re
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor
from preparando_datos import sum_into_column,split_and_sum,get_min_max,compute_average,sum_relevant_exp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline

from sklearn.base import BaseEstimator, TransformerMixin
from joblib import Parallel, delayed

#TODO polars



df_train=pd.read_csv('dataset/origen.csv')
genre_types = df_train['genres_x'].unique().tolist()

genre_types = [genre.split(",") if type(genre)==str else ['Missing'] for genre in genre_types]
genre_types = [item for sublist in genre_types for item in sublist]
genre_types = list(set(genre_types))



status_types = df_train['status'].unique().tolist()
status_types.remove(np.nan)
status_types.append('Missing')
status_types.append('Canceled')
""" 
for i, status in enumerate(status_types):
    status_types[i] = f"status_{status}" """
#take a 10% sample out of train
df_train.rename(columns={'Unnamed: 0':'exp'}, inplace=True)

df_test =  df_train.sample(frac=0.1, random_state=43)
#drop the sample from train
df_train.drop(df_test.index, inplace=True)

X_test = df_test.drop(["averageRating"], axis=1)
X = df_train.drop(["averageRating"], axis=1)
#Rename 'Unnamed: 0' to 'exp'


y = df_train["averageRating"]
y_test = df_test['averageRating']

del df_test,df_train



In [18]:

print(type(eval(X['production_companies'].iloc[15])[0]))


<class 'dict'>


In [19]:
X['runtimeMinutes'].replace(0, np.nan, inplace=True)
X['budget'].replace(0, np.nan, inplace=True)
X['revenue'].replace(0, np.nan, inplace=True)
X['titleType'].replace(0, np.nan, inplace=True)




In [20]:
print(X.production_countries.unique())

[nan "[{'iso_3166_1': 'IT', 'name': 'Italy'}]" '[]' ...
 "[{'iso_3166_1': 'FI', 'name': 'Finland'}, {'iso_3166_1': 'EE', 'name': 'Estonia'}, {'iso_3166_1': 'KE', 'name': 'Kenya'}]"
 "[{'iso_3166_1': 'IN', 'name': 'India'}, {'iso_3166_1': 'SE', 'name': 'Sweden'}, {'iso_3166_1': 'RS', 'name': 'Serbia'}, {'iso_3166_1': 'GR', 'name': 'Greece'}]"
 "[{'iso_3166_1': 'BE', 'name': 'Belgium'}, {'iso_3166_1': 'FR', 'name': 'France'}, {'iso_3166_1': 'PT', 'name': 'Portugal'}]"]


In [21]:

categorical_columns_with_nan = [
    c for c in X.columns if X[c].dtype == 'O' and X[c].isnull().any()]

categorical_columns_without_nan = [
    c for c in X.columns if X[c].dtype == 'O' and X[c].notnull().all()]



numerical_columns_with_nan = [
    c for c in X.columns if X[c].dtype != 'O' and X[c].isnull().any()]
numerical_columns_without_nan = [
    c for c in X.columns if X[c].dtype != 'O' and X[c].notnull().all()]

In [22]:
class TransformFromDict(BaseEstimator, TransformerMixin):
    #Voy a intentar explicar esto lo mejor que puedas
    def __init__(self, variables:list):
        #
        if type(variables) == str:
            variables = [variables]     
        self.variables = variables

    
    def fit(self, X, y=None):
        return self
    
    
    def transform(self, X):
        def get_str(x):
            #x is missing
            if x is np.nan or x is None or x == 'Missing' or x == 'nan' or x == '[]' or x == '':
                return ''
            x = eval(x)
            names = []
            if type(x)== list:
                for item in x:
                    if type(item)==dict:
                        name = item.get('name', '')
                    else:
                        name = ''
                    names.append(name)
            else:
                names.append('')
            return ','.join(names)
        for group in self.variables:
            X[group] = X[group].apply(get_str)
        return X
lista = []


In [23]:
from feature_engine.imputation import AddMissingIndicator, CategoricalImputer, MeanMedianImputer
from feature_engine.encoding import OneHotEncoder,RareLabelEncoder
from feature_engine.selection import  DropFeatures
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import FunctionTransformer
def interpolate_numericals(df:pd.DataFrame, group, category, method='linear'):
    # Fill in missing values using interpolation 
    interpolated = df.groupby(group)[category].apply(lambda x: x.interpolate(method=method))
    
    # Reset the index of the interpolated DataFrame
    interpolated = interpolated.reset_index(drop=True)
    
    # Replace the original column with the interpolated column
    df[category] = interpolated
    
    return df


""" 
# Initialize the fitted values dictionary
#interpolate_numericals.fitted_values = {}
for to_remove in ['budget','revenue']:
    try:
        numerical_columns_with_nan.remove(to_remove)
    except ValueError:
        pass """

def check(x):
    print(f"Number of missing values in titleType before categorical imputer: {x['titleType'].isna().sum()}")
    return x
class OverQuantileImputing(BaseEstimator, TransformerMixin):
    def __init__(self, variables:list, quantiles=4) -> None:
        super().__init__()
        self.quantiles = quantiles
        self.variables = variables
        self.thresholds = {}

    def fit(self, X:pd.DataFrame, _y=None):
        for var in self.variables:
            self.thresholds[var] = []
            for quantile in range(1, self.quantiles):
                self.thresholds[var].append(X[var].quantile(quantile/self.quantiles))
        return self

    def transform(self, X):
        for var in self.variables:
            X[f'{var}_quantile'] = 0
            for i, threshold in enumerate(self.thresholds[var]):
                X.loc[X[var] > threshold, f'{var}_quantile'] = i + 1
        return X

pipeline_preprocessing = Pipeline([
    ('turnToNan', FunctionTransformer(lambda x: x.assign(runtimeMinutes=x['runtimeMinutes'].replace(0, np.nan),
                                                          budget=x['budget'].replace(0, np.nan),
                                                          revenue=x['revenue'].replace(0, np.nan),
                                                          isAdult=x['isAdult'].apply(lambda x: 1 if x>1 else x),
                                                          ))),
    ('numericalMissingIndicator', AddMissingIndicator(variables=numerical_columns_with_nan)),
   ('categoricalMissingIndicator', AddMissingIndicator(variables=categorical_columns_with_nan)),
   ('quantileSignaler',OverQuantileImputing(variables=['runtimeMinutes','budget','revenue','numVotes'])),

    ('runtimeTransform', FunctionTransformer(lambda x: interpolate_numericals(x, 'titleType', 'runtimeMinutes'))),
    ('budgetTransform', FunctionTransformer(lambda x: interpolate_numericals(x, 'titleType', 'budget'))),
    ('revenueTransform', FunctionTransformer(lambda x: interpolate_numericals(x, 'titleType', 'revenue'))),
    ('endYearsTransform', FunctionTransformer(lambda X: X.assign(endYear=X.apply(lambda x: x['startYear'] if x['endYear'] == 0 else x['endYear'], axis=1)), validate=False)),
    ('categoricalImputer', CategoricalImputer(variables=categorical_columns_with_nan,fill_value='Missing')),
    ('fill_nans', FunctionTransformer(lambda x: x.assign(video = x['video'].replace('Missing',False),
                                                          tagline = x['tagline'].replace('Missing',''),
                                                          production_companies = x['production_companies'].replace('Missing',''),
                                                            production_countries = x['production_countries'].replace('Missing',''),
                                                          ))), 
    #('meanMedianImputer', MeanMedianImputer(imputation_method='median', variables=numerical_columns_with_nan)),
    ('getSTRsFromDicts', TransformFromDict(variables=['production_companies','production_countries'])),
    #('tvCategories',RareLabelEncoder(tol=0.01,n_categories=6, variables=['titleType'],replace_with='Other_TV')),
])






In [24]:
#la principal diferencia entre este y el anterior es que el anterior no divide por columnas, sino que muestran unicamente la experiencia en el tag relevante
class GetExpDict(BaseEstimator, TransformerMixin):
    #Voy a intentar explicar esto lo mejor que puedas
    def __init__(self, group:list, categories:list,targets:list=[''],simple_only:list = [],simple_targets:list = []):
        super().__init__()
        if type(categories) == str:
            categories = [categories]
        if type(targets) == str:
            targets = [targets]
        group.extend(simple_only)
        targets.extend(simple_targets)
        self.group = group
        self.categories = {category: [] for category in categories}
        self.simple_only = simple_only
        self.targets = targets
        self.simple_targets = simple_targets

    def process_group_category_target(self,group, category, target, X):
        #print('Processing group:',group, category, target)
        categorias = self.categories[category]
        grupo = X.groupby(group).apply(lambda x: [(g, v) for g, v in zip(x[category], x[target] if target== 'exp' else x[category].value_counts())])
        #grupo.to_json(f'grupo_{group}_{category}_{target}.json')
        grupo = grupo.to_dict()
        return (group, grupo, categorias, target)


    def fit(self, X, y=None):
        for category in self.categories:
            categorias = X[category].unique().tolist()
            categorias = [x.split(',') if type(x) == str else [] for x in categorias]
            categorias = [item for sublist in categorias for item in sublist]
            categorias = list(set(categorias))
            self.categories[category] = categorias
        
        print(self.group, self.categories, self.targets)
        print('creating parallel fitting job')
        results = Parallel(n_jobs=-1)(delayed(self.process_group_category_target)(group, category, target, X) for group in self.group for category in self.categories for target in self.targets)
        grupos = [result for result in results]
        print('done')
        self.diccionarios = Parallel(n_jobs=-1)(delayed(self._fit_group)(group, grupo, categorias,target) for group, grupo, categorias,target in grupos)
        print('done')
        return self
    
    def _fit_group(self, group, grupo, categorias, target):
        # Create a dictionary with directors as keys and their shows as values
        
        dict_limpio = {}
        for sub_grupo, shows in grupo.items():
            for dir in sub_grupo.split(','):
                dict_limpio[dir] = shows

        # Initialize a dictionary to count categories
        cat_counts = {}
        for sub_grupo in dict_limpio.keys():
            cat_counts[sub_grupo] = {}
            for cat in categorias:
                cat_counts[sub_grupo][cat] = [0, 0]

        # Update the count for each category
        for sub_grupo, values in dict_limpio.items():
            for v in values:
                cat = v[0]
                count = v[1]
                if cat in cat_counts[sub_grupo]:
                    cat_counts[sub_grupo][cat][0] += count
                    cat_counts[sub_grupo][cat][1] += 1

        # Compute the average for each category and store it in authors_xp
        authors_xp = {}
        for sub_grupo, cat_dict in cat_counts.items():
            authors_xp[sub_grupo] = {}
            for cat, counts in cat_dict.items():
                authors_xp[sub_grupo][cat] = compute_average(counts)
        
        if '0' in authors_xp:
            del authors_xp['0']
        if 'Missing' in authors_xp:
            del authors_xp['Missing']
        if '' in authors_xp:
            del authors_xp['']
        return group, authors_xp, categorias, target
    
    def transform(self, X):
        # Define a helper function for parallel processing
        def process_group(group, diccionario, categorias, target):
            new_columns = {}
            if group not in self.simple_only and target not in self.simple_targets:
                new_columns.update({
                    f'{group}_{target}_{category}': X[group].apply(sum_into_column, args=(diccionario, category,))
                    for category in categorias
                })
            new_columns.update({
                f'{group}_{target}_min': X[group].apply(get_min_max, args=(diccionario, 0, categorias,)),
                f'{group}_{target}_max': X[group].apply(get_min_max, args=(diccionario, 1, categorias,)),
                f'{group}_{target}_total': X[group].apply(sum_into_column, args=(diccionario, 'total',)),
                f'{group}_{target}_relevant': X.apply(sum_relevant_exp, args=(diccionario,group,target), axis=1),
            })
            return new_columns

        # Run the helper function in parallel
        results = Parallel(n_jobs=1)(delayed(process_group)(group, diccionario, categorias, target) for group, diccionario, categorias, target in self.diccionarios)

        # Combine the results
        new_columns = {k: v for result in results for k, v in result.items()}

        X = pd.concat([X, pd.DataFrame(new_columns)], axis=1)
        return X





""" pipeline_exp = Pipeline([
    
]) """


' pipeline_exp = Pipeline([\n    \n]) '

In [25]:
def okay(x):
    print("Okay")
    return x
def binarize_genres(X):
    mlb = MultiLabelBinarizer()
    print('0')
    binarized_genres = mlb.fit_transform(X['genres_x'].str.split(','))
    df_binarized_genres = pd.DataFrame(binarized_genres, columns=mlb.classes_)
    
    print('1')
    for column in genre_types:
        if column not in df_binarized_genres.columns:
            df_binarized_genres[column] = 0
    print('2')
    df_binarized_genres = pd.concat([X, df_binarized_genres], axis=1)
    print('3')
    return df_binarized_genres

#TODO: Comparar a ver si es mejor ni usar order, season y episodeNumber
class GetDetailedExp(BaseEstimator, TransformerMixin):

    def __init__(self, group:dict, targets:dict):
        super().__init__()
        self.group = group
        self.targets = targets

    def process_group_target(self,group, target, X):
        #print('Processing group:',group, category, target)
        objetivos = {}
        if self.targets[target]==0:
            objetivos[f"{group}_{target}"] = X.groupby(group).count()[target].to_dict()

        else:
            for sub_target in self.targets[target]:
                if sub_target:
                    mask = X[target] == sub_target
                    if mask.any():
                        objetivos[f"{group}_{target}_{sub_target}"] = X[mask].groupby(group).count()[target].to_dict()
                    else:
                        print(f"No rows where target == {sub_target}")
                else:
                    objetivos[f"{group}_{target}_other"] = X[~X[target].isin(self.targets[target][:-1])].groupby(group).count()[target].to_dict()
        
        for grupo,diccionario in objetivos.items():
            objetivos[grupo] = {
                "datos" : split_and_sum(diccionario),
                "grupo": group,
            }
            if '0' in objetivos[grupo]["datos"]:
                del objetivos[grupo]["datos"]['0']
            if 'Missing' in objetivos[grupo]["datos"]:
                del objetivos[grupo]["datos"]['Missing']
            if '' in objetivos[grupo]["datos"]:
                del objetivos[grupo]["datos"]['']

        
        return objetivos


    def fit(self, X, y=None):
        print(self.group, self.targets)
        print('creating parallel fitting job')
        grupos = dict()
        results = Parallel(n_jobs=-1)(delayed(self.process_group_target)(group, target, X) for group in self.group for target in self.targets)
        grupos.update({group: result for dictionary in results for group, result in dictionary.items()})
        print('done')
        self.diccionarios = grupos
        return self

    def transform(self,X):

        def process_group(nombre,diccionario):
            return {nombre:X[diccionario['grupo']].apply(sum_into_column, args=(diccionario['datos'],))}
            
        results = Parallel(n_jobs=-1)(delayed(process_group)(nombre, diccionario) for nombre,diccionario in self.diccionarios.items())

        # Combine the results
        new_columns = {k: v for result in results for k, v in result.items()}

        X = pd.concat([X, pd.DataFrame(new_columns)], axis=1)
        return X
    


pipeline = Pipeline([('preprocessing',pipeline_preprocessing),
    ('set_exp_dict_genres', GetExpDict(group=['directors',"writers"],
                                    categories=['genres_x','titleType'],
                                    simple_only=['production_companies','production_countries'],
                                    targets=['exp'],
                                    simple_targets=['runtimeMinutes','numVotes'])),
    ('set_detailed_exp',GetDetailedExp(group=['directors',"writers"],
                                       targets={'seasonNumber':0,'episodeNumber':0,'titleType':['movie',None]})),

    ('binarizeGenres', FunctionTransformer(binarize_genres)),

    ('removeEmptyRows', FunctionTransformer(lambda x: x.dropna(subset=['directors']))),

    
    ('oneHotEncoder', OneHotEncoder(variables=['titleType',"status"])),
    
    ('get_all_companies',FunctionTransformer(lambda x: x.assign(production_companies=x['production_companies'].apply(lambda x: len(x.split(','))),
                                                                production_countries=x['production_countries'].apply(lambda x: len(x.split(','))),
                                                                directors=x['directors'].apply(lambda x: len(x.split(','))),
                                                                writers=x['writers'].apply(lambda x: len(x.split(','))),
                                                                tagline=x['tagline'].apply(len),
                                                                video = x['video'].astype(int),
                                                                budget = x['budget'].fillna(0),
                                                                ))),

    ('removeCategorical',FunctionTransformer(lambda x: x.drop(x.select_dtypes(include=['object']).columns, axis=1))), 
    ('removeExpColumn',FunctionTransformer(lambda x: x.drop(['exp'], axis=1))),


    ])

In [26]:
pipeline.fit(X,y) #4m 30.1

['directors', 'writers', 'production_companies', 'production_countries'] {'genres_x': ['Family', 'Talk-Show', 'History', 'Adventure', 'Action', 'Sport', 'Mystery', 'Comedy', 'Animation', 'Fantasy', 'Reality-TV', 'Musical', 'Short', 'Missing', 'Drama', 'Music', 'Adult', 'Film-Noir', 'Thriller', 'Romance', '0', 'Horror', 'Biography', 'Western', 'News', 'Sci-Fi', 'Game-Show', 'Documentary', 'War', 'Crime'], 'titleType': ['tvSeries', 'videoGame', 'tvMovie', 'tvMiniSeries', 'tvSpecial', 'video', 'tvEpisode', 'short', 'tvShort', 'movie']} ['exp', 'runtimeMinutes', 'numVotes']
creating parallel fitting job
done
done
['directors', 'writers'] {'seasonNumber': 0, 'episodeNumber': 0, 'titleType': ['movie', None]}
creating parallel fitting job
done
0
1
2
3


Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('turnToNan',
                                  FunctionTransformer(func=<function <lambda> at 0x00000288BEC7A160>)),
                                 ('numericalMissingIndicator',
                                  AddMissingIndicator(variables=['runtimeMinutes',
                                                                 'seasonNumber',
                                                                 'episodeNumber',
                                                                 'ordering',
                                                                 'isOriginalTitle',
                                                                 'budget',
                                                                 'popularity',
                                                                 'revenue',
                                                                 'runtime'])),
                                 ('categoricalMissingIndicator',
                                  AddMissin...
                 FunctionTransformer(func=<function <lambda> at 0x00000289306EDB20>)),
                ('oneHotEncoder',
                 OneHotEncoder(variables=['titleType', 'status'])),
                ('get_all_companies',
                 FunctionTransformer(func=<function <lambda> at 0x0000028930753380>)),
                ('removeCategorical',
                 FunctionTransformer(func=<function <lambda> at 0x00000289307531A0>)),
                ('removeExpColumn',
                 FunctionTransformer(func=<function <lambda> at 0x0000028930752C00>))])

In [27]:
#X_pred = pd.read_csv('dataset/testear.csv').drop(['Unnamed: 0'],axis=1)
jjjj = pipeline.transform(X)

#293 3m 42.8

KeyboardInterrupt: 

In [ ]:
#adding the target
""" jjjj['averageRating'] = y
jjjj.to_csv('dataset/transformedd.csv') """

In [ ]:
#X_test2 = pipeline.transform(X_test)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
import numpy as np
from xgboost import XGBRegressor
# import cupy as cp
# import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split



In [ ]:
regr = XGBRegressor(n_estimators=1100,eta=0.05,
                        max_depth=20,gamma = 0.20,
                        colsample_bytree = 0.7,colsample_bylevel=0.7,colsample_bynode=0.8,
                        tree_method = 'hist',
                        max_cached_hist_node=262144) # type: ignore
#regr = XGBRegressor(n_estimators=1200, eta=0.35, max_depth=7, multi_strategy="multi_output_tree", min_child_weight=1, subsample=1, colsample_bytree=1, gamma=0, alpha=0)
print("Training a XGBRegressor")
regr.fit(jjjj, y)
print("Finished training the XGBRegressor")
#Tarda 49 minutos con esos settings 4565
regr.score(pipeline.transform(X_test),y_test)



In [ ]:
from lightgbm import LGBMRegressor

lrerg = LGBMRegressor(n_estimators=5000,learning_rate=0.2)
lrerg.fit(pipeline.transform(X), y)
lrerg.score(pipeline.transform(X_test),y_test)


In [ ]:
#Grid search
from sklearn.model_selection import RandomizedSearchCV
param_grid = {
    'n_estimators': [1000, 1500, 2000],
    'max_depth': [6, 8, 10,17,19,20],
    'gamma':[0.1,0.2,0.3,0.5,0.7,1],
    'min_child_weight': [1,2,3,4],
}

regr = XGBRegressor()
grid_search = RandomizedSearchCV(regr, param_grid,n_iter=100, cv=5, scoring='neg_mean_squared_error')

grid_search.fit(jjjj, y)
print(grid_search.best_params_)
print(grid_search.best_score_)
""" 
regr.fit(jjjj, y)
regr.score(X_test,y_test) #8 ~ 0.4165 # 5000 y 4 = 0.3958 """

In [ ]:

#X_test = pipeline.transform(X_test)
print("Predicting")
score = regr.score(X_test, y_test) #4797

print(f"R^2 score on testing data: {score:.4f}") 
# RMSE
from sklearn.metrics import mean_squared_error

y_pred = regr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))
#print(set(X_pred.columns))

In [ ]:
X_pred = pd.read_csv('dataset/testear.csv').rename(columns={'Unnamed: 0':'exp'})

X_pred = pipeline.transform(X_pred)
Yest = regr.predict(X_pred).clip(1, 10)
salida = pd.DataFrame(data={"averageRating": Yest})
salida.index = X_pred.index
salida.to_csv("predicciones/pred40.csv", sep=',',index=True,  index_label='Id')